In [1]:
import sys
from pathlib import Path

In [2]:
import storysniffer
import pandas as pd
import altair as alt

In [3]:
this_dir = Path("__file__").parent.absolute()

In [4]:
sys.path.append(str(this_dir.parent / "newshomepages"))

In [5]:
extracts_dir = this_dir.parent / "extracts" / "csv"

In [6]:
analysis_dir = this_dir.parent / "_analysis"

In [7]:
df = pd.read_csv(analysis_dir / "drudge-hyperlinks.csv", dtype=str)

In [8]:
df = df[(~pd.isnull(df.url)) & (~pd.isnull(df.text))].copy()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62696 entries, 0 to 62972
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             62696 non-null  object
 1   url              62696 non-null  object
 2   site_handle      62696 non-null  object
 3   item_identifier  62696 non-null  object
 4   file_timestamp   62696 non-null  object
 5   file_url         62696 non-null  object
dtypes: object(6)
memory usage: 3.3+ MB


In [10]:
df.head()

,text,url,site_handle,item_identifier,file_timestamp,file_url
0,POLL: Putin popularity UP since start of confl...,https://extra.cw/putins-popularity-up-since-st...,drudge,drudge-2022,2022-03-31 14:42:51,https://archive.org/download/drudge-2022/drudg...
1,Lied to by advisers?,https://nypost.com/2022/03/30/putin-being-lied...,drudge,drudge-2022,2022-03-31 14:42:51,https://archive.org/download/drudge-2022/drudg...
2,UK spy chief says soldiers disobey orders...,https://apnews.com/article/russia-ukraine-puti...,drudge,drudge-2022,2022-03-31 14:42:51,https://archive.org/download/drudge-2022/drudg...
3,"Russia drafts 134,500 conscripts...",https://www.usnews.com/news/world/articles/202...,drudge,drudge-2022,2022-03-31 14:42:51,https://archive.org/download/drudge-2022/drudg...
4,Troops Suffer 'Acute Radiation Sickness' After...,https://news.yahoo.com/russian-troops-suffer-a...,drudge,drudge-2022,2022-03-31 14:42:51,https://archive.org/download/drudge-2022/drudg...


In [11]:
unique_df = df.groupby(["text", "url"]).size().rename("count").reset_index()

In [12]:
sniffer = storysniffer.StorySniffer()

/home/palewire/.local/share/virtualenvs/news-homepages-Qlfa7zLV/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/palewire/.local/share/virtualenvs/news-homepages-Qlfa7zLV/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator FunctionTransformer from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/palewire/.local/share/virtualenvs/news-homepages-Qlfa7zLV/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying t

In [13]:
unique_df['is_story'] = unique_df.apply(lambda x: sniffer.guess(x['url'], x['text']), axis=1)

In [14]:
unique_df.is_story.value_counts()

True     8582
False     315
Name: is_story, dtype: int64

In [15]:
unique_df.is_story.value_counts(normalize=True)

True     0.964595
False    0.035405
Name: is_story, dtype: float64

In [16]:
unique_df.to_csv("./drudge-hyperlink-counts.csv", index=False)